In [1]:
import time, datetime
import codecs
import json
from TwitterAPI import TwitterAPI
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#TwitterAPI Settings
api = TwitterAPI('OOheRS0hK5FQaj6iL75XTILke', 'aDeZ9i3dZ18KjtDiEcNf683hwjKC4MLyAR3v1f1FqHiW3QdNEz', '831073699411275776-7HDtuctKY7orzuEIRPhEkChTgo2JJ5n', 'txslgzywJrEMmy6zi2e8AdpikcqsWo6k9hueSJMibSeZx')

#Open webdriver Chrome
#Use location to your chromedriver.exe
browser = webdriver.Chrome("chromedriver.exe")

#search url
#a
browser.get("https://twitter.com/search?f=tweets&vertical=default&q=diare%20OR%20imodium%20OR%20diapet%20OR%20%22sakit%20perut%22%20OR%20mules%20since%3A2018-12-08%20until%3A2018-12-15&l=id&src=spxr")
#b
# browser.get("https://twitter.com/search?f=tweets&vertical=default&q=mencret%20OR%20menceret%20OR%20salmonella%20OR%20%22e%20coli%22%20OR%20oralit%20since%3A2018-09-01%20until%3A2018-12-31&l=id&src=spxr")

lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match = False
while (match==False):
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount == lenOfPage:
        match=False

KeyboardInterrupt: 

In [ ]:
#Open session to write
outputFile = codecs.open("dia des a2.csv", "w+", "utf-8")
outputFile.write('username,date,time,text,tweetID,placeID,daerah,lokasi,longitude,latitude\n')

#tweet_contents 
tweet_contents = browser.find_elements_by_class_name("js-stream-tweet")

index = 0

for tweet in tweet_contents[index:]:
    #Tweet text
    text = tweet.find_element_by_css_selector("div.js-tweet-text-container > p").text.replace('\n',' ').replace(',', ' ')
    #Username
    username = '@'+tweet.find_element_by_css_selector("div.stream-item-header > a > span.username > b").text
    #Date of tweet
    dateInt = int(tweet.find_element_by_css_selector("div.stream-item-header > small > a > span").get_attribute('data-time'))
    dateFormat = datetime.datetime.fromtimestamp(dateInt)
    #Tweet ID
    tweetID = int(tweet.get_attribute("data-tweet-id"))
    
    r = api.request('statuses/show/:%d' % tweetID)
    if r.status_code != 200:
        print('false status')
        print(r.status_code)
        print(username)
        print(tweetID)
    else:
        for item in r.get_iterator():
            #Place ID
            if item['place'] != None:
                placeID = item['place']['id']
                r = api.request('geo/id/:%s' % placeID)

                if r.status_code != 200:
                    print('false place')
                    print(r.status_code)
                    print(placeID)
                else:
                    json_var = r.json()
                    #name
                    daerah = json_var['name']
                    #fullname
                    lokasi = json_var['full_name'].replace(',', '')
                    #longitude
                    longitude = json_var['centroid'][0]
                    #latitude
                    latitude = json_var['centroid'][1]
            else:
                placeID = ''
                daerah = ''
                lokasi = ''
                longitude = ''
                latitude = ''
        #test
        print(index)
        print('username: ' + username)
        print('date: ' + dateFormat.strftime("%Y-%m-%d"))
        print('time: ' + dateFormat.strftime("%H:%M"))
        print('text: ' + text)
        print('tweet id: %s ' % tweetID)
        print('place id: %s ' % placeID)
        print('daerah: ' + daerah)
        print('lokasi: ' + lokasi)
        print('longitude: %s ' % longitude)
        print('latitude: %s ' % latitude)
        print()
        index = index + 1
        outputFile.write('%s, %s, %s, %s, %s, %s, %s, %s, %s, %s\n' % (username, dateFormat.strftime("%Y-%m-%d"), dateFormat.strftime("%H:%M"), text, tweetID, placeID, daerah, lokasi, longitude, latitude))

0
username: @choadachiu
date: 2018-12-15
time: 06:58
text: SORRY AKU BACANYA BER*K...   abisan di tweet sebelumnya kamu mules kak 
tweet id: 1073728998868377600 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

1
username: @bertopikuning
date: 2018-12-15
time: 06:56
text: Rutinitas setiap pagi: mules bisa bolak balik kamar mandi sampe 5 kali
tweet id: 1073728382834069504 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2
username: @navynth
date: 2018-12-15
time: 06:54
text: Sakit perut weh padahal udah boker
tweet id: 1073728007930404864 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

3
username: @andrewsitorus
date: 2018-12-15
time: 06:53
text: Memasuki waktu indonesia bagian perut lapar dan mules bebarengan. 
tweet id: 1073727603976986624 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

4
username: @_ssscar
date: 2018-12-15
time: 06:50
text: Aku sakit perut. Apakah ini efek finale voltron.
tweet id: 1073727008230727681 
place id:  
daerah: 
lokasi: 
lo

38
username: @madilarzkyn
date: 2018-12-15
time: 05:22
text: Kadang gabisa bedain mana perut keroncongan laper/ perut mules kebelet -_-
tweet id: 1073704712355012608 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

39
username: @abidinode09
date: 2018-12-15
time: 05:19
text: Wah bahaya nih perut  Makan langsung sakit perut Kalau tak makan lapar pulak 
tweet id: 1073703941001498625 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

40
username: @lovinaulia
date: 2018-12-15
time: 05:18
text: Utk yg cewek nih kalau sakit perut pas pms biasanya diapain? Minum obat atau gimana? — Cuma dibawa tidur aja karna sakitnya tuh kandh maendh kadang hmm 
tweet id: 1073703740522168320 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

41
username: @aimanomaar
date: 2018-12-15
time: 05:14
text: aq dh la susah tidur  bila dh tidur bole terjaga plak sbb sakit perut. kejap lagi kena bgun awal pagi . mata dh sembab ni. siol laaaaa
tweet id: 1073702769440116736 
place id:  
daerah: 
loka

76
username: @JAN_0899
date: 2018-12-15
time: 02:24
text: Aduii..sakit perut aq gelak
tweet id: 1073659933252476928 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

77
username: @rizkiesaskia
date: 2018-12-15
time: 02:15
text: Terbangun karena mules itu....kzl
tweet id: 1073657844249939968 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

78
username: @kyztali
date: 2018-12-15
time: 02:12
text: ngakak mules kasian bgtt
tweet id: 1073657086628614144 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

79
username: @sukabudiraharja
date: 2018-12-15
time: 02:08
text: Jam segini perut mules 
tweet id: 1073655974395662336 
place id: 48f3a332643193ef 
daerah: Magelang Utara
lokasi: Magelang Utara Indonesia
longitude: 110.2202591273853 
latitude: -7.46148075 

80
username: @sok_keren
date: 2018-12-15
time: 02:06
text: Seks di luar pernikahan itu bagaikan kentut  kalo ditahan bakal sakit perut  tapi kalo di lepas malu menjemput!!
tweet id: 1073655538418835457 
place id:  
da

114
username: @jefriniichol
date: 2018-12-15
time: 01:05
text: Tar makin sakit perut nya
tweet id: 1073640010841083905 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

115
username: @ashraffitri41
date: 2018-12-15
time: 01:04
text: Sakit perut ni dah tak boleh buat bincang ke ape
tweet id: 1073639877403598849 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

116
username: @jgchaey
date: 2018-12-15
time: 01:04
text: aq mules bacanyah
tweet id: 1073639816145788928 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

117
username: @paisukidesu
date: 2018-12-15
time: 01:02
text: Knp si kudu mules jam segini
tweet id: 1073639487744311296 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

118
username: @xyonie
date: 2018-12-15
time: 01:00
text: kok perut gue mules lg ya
tweet id: 1073638911073693696 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

119
username: @NikFikriZharif
date: 2018-12-15
time: 01:00
text: Sakit perut jangan salahkan bontot  cuba tanya p

152
username: @Josue_Andrean
date: 2018-12-15
time: 00:19
text: Wkwkwkkkkkk mules mesti
tweet id: 1073628583786991616 
place id: 42ee1fbd9f9c5a0f 
daerah: Gajahmungkur
lokasi: Gajahmungkur Indonesia
longitude: 110.40119019297924 
latitude: -7.01057935 

153
username: @hiiamrty
date: 2018-12-15
time: 00:16
text: Tapi tadulu...gua sakit perut
tweet id: 1073627734121754624 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

154
username: @aquabrome
date: 2018-12-15
time: 00:11
text: Positif diare negatif diare tms ms : (
tweet id: 1073626613604970496 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

155
username: @tamanavara
date: 2018-12-15
time: 00:08
text: Mules aku  Teh. T A T
tweet id: 1073625841848901632 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

156
username: @vertixgnouss
date: 2018-12-15
time: 00:07
text: nyesel makan indomi 3 bungkus  sampe skrg sakit perut:(
tweet id: 1073625629650649088 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

157
user

191
username: @yeownwoo
date: 2018-12-14
time: 23:27
text: SAKIT PERUT? MINUM BAYGOOONNN  EH GUA JADI NUNA YA? SINI SALIM SAMA NUNA 
tweet id: 1073615590257057793 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

192
username: @chocotopmcd
date: 2018-12-14
time: 23:27
text: Napa si mules jam segini -______-
tweet id: 1073615462649323522 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

193
username: @irenedien
date: 2018-12-14
time: 23:27
text: Jam 8 uda teler. Ketiduran sambil badan ditumpuki bantal bantal. Bangun jam 11 lebih  mendapati suami belum pulang heboh sendiri. Mendadak perut mules. Ga boleh sedih  ga boleh sedih. Pukpuk perut sayang
tweet id: 1073615409729789952 
place id: 31569f33fbe4e733 
daerah: Lowokwaru
lokasi: Lowokwaru Indonesia
longitude: 112.60684962877417 
latitude: -7.943315 

194
username: @CEKAHABODO
date: 2018-12-14
time: 23:25
text: Ke wc sana  sakit perut malah laporan cih
tweet id: 1073615093835780097 
place id:  
daerah: 
lokasi: 
longitude: 

228
username: @nisyanis
date: 2018-12-14
time: 22:50
text: Jadi gini alergi kacang  sakit perut karena gas. Kembung gak jelah heu
tweet id: 1073606247927042048 
place id: 04f4e198a524dc50 
daerah: Kembaran
lokasi: Kembaran Indonesia
longitude: 109.29388971769707 
latitude: -7.41493305 

229
username: @ssocholate
date: 2018-12-14
time: 22:50
text: ya beda klo kmu lg diare mukanya beda sma aku :3
tweet id: 1073606153731485696 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

230
username: @adakerinduan
date: 2018-12-14
time: 22:49
text: NGGAK.   IH LAGI SAKIT PERUT AJA
tweet id: 1073605968666099712 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

231
username: @melsiadara
date: 2018-12-14
time: 22:48
text: Ga ah sakit perut nanti aku 
tweet id: 1073605781465878529 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

232
username: @TopeRendusara
date: 2018-12-14
time: 22:47
text: Terdenger suara pokalis denasip dari kamar asisten rt  langsung perut mules kentut kentut 


269
username: @MaureenOliviaL
date: 2018-12-14
time: 22:13
text: Muridku yang masih kelas 3 SD  kalau pup masih harus ditungguin sama babysitternya. Akhir-akhir ini dia suka sakit perut. Selalu buka celana depan di ruang tamu (tempat belajar). Terus tadi dia nyuruh gue liat 鸟. Ya Allah Bapa.
tweet id: 1073596969229070336 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

270
username: @bungakeciI
date: 2018-12-14
time: 22:13
text: "Aduh. . . Mules. . . Makan apa aja sih aku hari ini. ."
tweet id: 1073596951139016704 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

271
username: @treafsure
date: 2018-12-14
time: 22:12
text: nonton asmr got7 gue sakit perut  ngakak
tweet id: 1073596531209494528 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

272
username: @Muhammadanggik
date: 2018-12-14
time: 22:11
text: Orang ganteng sakit perut beraknya tai juga.
tweet id: 1073596338149908480 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

273
username: @gumajjin
date: 

308
username: @serendiviety
date: 2018-12-14
time: 21:43
text: mules kan mana aer lagi mati
tweet id: 1073589220743163904 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

309
username: @moonfaiyr
date: 2018-12-14
time: 21:43
text: YANG PERFORM SIAPA YG MULES SIAPA 
tweet id: 1073589219191214082 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

310
username: @nrdnsh_
date: 2018-12-14
time: 21:42
text: Weyh sakit perut lah
tweet id: 1073589110005161985 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

311
username: @MO0NCATS
date: 2018-12-14
time: 21:42
text: HSHSHSH SAKIT PERUT WEYH CEPAT WEYH
tweet id: 1073589093030719488 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

312
username: @pchybngtans
date: 2018-12-14
time: 21:42
text: jdi mules gini ampun dah
tweet id: 1073588946582396929 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

313
username: @likedust1104
date: 2018-12-14
time: 21:41
text: SAKIT PERUT
tweet id: 1073588862234972161 
place id:  

In [4]:
outputFile = codecs.open("dia dec a2 2115.csv", "w+", "utf-8")
outputFile.write('username,date,time,text,tweetID,placeID,daerah,lokasi,longitude,latitude\n')

#tweet_contents 
tweet_contents = browser.find_elements_by_class_name("js-stream-tweet")

index = 2115

for tweet in tweet_contents[index:]:
    #Tweet text
    text = tweet.find_element_by_css_selector("div.js-tweet-text-container > p").text.replace('\n',' ').replace(',', ' ')
    #Username
    username = '@'+tweet.find_element_by_css_selector("div.stream-item-header > a > span.username > b").text
    #Date of tweet
    dateInt = int(tweet.find_element_by_css_selector("div.stream-item-header > small > a > span").get_attribute('data-time'))
    dateFormat = datetime.datetime.fromtimestamp(dateInt)
    #Tweet ID
    tweetID = int(tweet.get_attribute("data-tweet-id"))
    
    r = api.request('statuses/show/:%d' % tweetID)
    if r.status_code != 200:
        print('false status')
        print(r.status_code)
        print(username)
        print(tweetID)
    else:
        for item in r.get_iterator():
            #Place ID
            if item['place'] != None:
                placeID = item['place']['id']
                r = api.request('geo/id/:%s' % placeID)

                if r.status_code != 200:
                    print('false place')
                    print(r.status_code)
                    print(placeID)
                else:
                    json_var = r.json()
                    #name
                    daerah = json_var['name']
                    #fullname
                    lokasi = json_var['full_name'].replace(',', '')
                    #longitude
                    longitude = json_var['centroid'][0]
                    #latitude
                    latitude = json_var['centroid'][1]
            else:
                placeID = ''
                daerah = ''
                lokasi = ''
                longitude = ''
                latitude = ''
        #test
        print(index)
        print('username: ' + username)
        print('date: ' + dateFormat.strftime("%Y-%m-%d"))
        print('time: ' + dateFormat.strftime("%H:%M"))
        print('text: ' + text)
        print('tweet id: %s ' % tweetID)
        print('place id: %s ' % placeID)
        print('daerah: ' + daerah)
        print('lokasi: ' + lokasi)
        print('longitude: %s ' % longitude)
        print('latitude: %s ' % latitude)
        print()
        index = index + 1
        outputFile.write('%s, %s, %s, %s, %s, %s, %s, %s, %s, %s\n' % (username, dateFormat.strftime("%Y-%m-%d"), dateFormat.strftime("%H:%M"), text, tweetID, placeID, daerah, lokasi, longitude, latitude))

2115
username: @HasfiqIkqmal
date: 2018-12-13
time: 08:14
text: Sakit perut 
tweet id: 1073023350555926529 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2116
username: @nadenaayy
date: 2018-12-13
time: 08:13
text: Gue mules   tp cape nongkrong. Hm
tweet id: 1073023071588474880 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2117
username: @overbakedbeans
date: 2018-12-13
time: 08:13
text: yang pasti aku sakit perut sampai ni aku terus naik gi toilet
tweet id: 1073022967808815104 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2118
username: @Sf_Aprianty
date: 2018-12-13
time: 08:12
text: Kenapa kebiasaan banget tiap abis sarapan pagi pasti langsung mules:(( sayang capek ngunyah 
tweet id: 1073022807842246656 
place id: b944c273d48123b9 
daerah: Cipeundeuy
lokasi: Cipeundeuy Indonesia
longitude: 107.59861450517272 
latitude: -6.5250035 

2119
username: @FyraIsmail91
date: 2018-12-13
time: 08:12
text: Haihh sakit perut. 
tweet id: 1073022799185272832 
place i

2151
username: @slashwirez
date: 2018-12-13
time: 07:51
text: Haha i feel you. Kemana2 bareng2 itu ngabisin energi.  Kalo lg butuh my space biasanya gw ke toilet. Sehari bisa 3-5x ke toilet.  Suka ditanya knp sering ke toilet lama2. Gw bilang gw gampang mules jadi pgn eek mulu.  Padahal gw ditoilet cuma duduk doang. Recharge.
tweet id: 1073017533588860928 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2152
username: @KORANBASI
date: 2018-12-13
time: 07:51
text: tidak kuat menahan rasa mules di angkutan umum  wanita hamil ini berhenti di toilet umum.
tweet id: 1073017442593497088 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2153
username: @noegroho_edy
date: 2018-12-13
time: 07:50
text: Mendung(sedikit rintik)  Atis-atis  Masuk Siang  Tidur lagi syahdu kali yaaa.... . . Tapi sayang  kenapa mules datang ketika cuaca mendukung seperti ini? Coba pengertian dikit (misale) 
tweet id: 1073017167040339968 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2154
usern

2187
username: @unconcsious
date: 2018-12-13
time: 07:13
text: gue mules bgt mau uji komprehensif
tweet id: 1073008089886945280 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2188
username: @Risdiana_sptr
date: 2018-12-13
time: 07:13
text: Langsung mules ni perut
tweet id: 1073007874282971136 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2189
username: @ysmnshhda
date: 2018-12-13
time: 07:12
text: Still lagi sakit perut tak hilang hilang ni kenapa 
tweet id: 1073007636361035776 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2190
username: @MissNyemil
date: 2018-12-13
time: 07:10
text: Yah udah rapi rapi  mau saraan tapi takut muleh nih ... Tenang ajaah MissNyemil punya cemilan anti mules
tweet id: 1073007263239950336 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2191
username: @febrinafeee
date: 2018-12-13
time: 07:09
text: Nggk suka klo diare giniiii :((( lemexzz
tweet id: 1073006987951038464 
place id:  
daerah: 
lokasi: 
longitude:  
latitud

2225
username: @oliviakrst
date: 2018-12-13
time: 06:26
text: Hari ini rencananya bangun siang. Njuk gagal  wetengku mules. Ncen ra ditakdirke tangi awan pas masa liburan jebul. Nek kuliah  tangi awan iso banget. Hah menungso
tweet id: 1072996238264418304 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2226
username: @OldyFahlovi73
date: 2018-12-13
time: 06:26
text: Kalo rasa sakit perut gw setiap pagi bisa di convert jadi 1 dollar mungkin sekarang gw udah jadi orang terkaya di dunia
tweet id: 1072996025059631105 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2227
username: @krucill_
date: 2018-12-13
time: 06:25
text: Sudah tiga hari ini tiap bangun pagi  perut selalu mules
tweet id: 1072995766493270018 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2228
username: @anggundwe
date: 2018-12-13
time: 06:24
text: Ini diare dah 2 minggu ga kelar kelar  ya allah menyiksa bgt 
tweet id: 1072995754673750018 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2

2263
username: @syieroxx
date: 2018-12-13
time: 04:20
text: Arghh sakit perut
tweet id: 1072964410858102784 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2264
username: @ValihAlfi
date: 2018-12-13
time: 04:13
text: Yang bisa membangunkanku langsung dari tidur :  Alarm   Mules 
tweet id: 1072962653134323712 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2265
username: @putrizdi
date: 2018-12-13
time: 04:02
text: Pepagi dah sakit perut pasaipa 
tweet id: 1072959902329393153 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2266
username: @ddaengjap
date: 2018-12-13
time: 03:53
text: Jadi #stupidmetoday adalah minum kopi jam segini trs nyesel jd sakit perut pake banget banget banget.
tweet id: 1072957685539041280 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2267
username: @NAY42210755
date: 2018-12-13
time: 03:49
text: Bikin Sakit Perut! 10 Anime Comedy Paling Lucu DiTahun 2018 http://news.masjmzs.com//index/xnews/index.html?uuid=f25df86b866e6dc00ea

...

6907
username: @dangitsuke
date: 2018-12-08
time: 09:52
text: %. Bentar. Saya laper sama mules. Plot dibalas nanti  ya.
tweet id: 1071236067871735809 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6908
username: @Famous_Teme
date: 2018-12-08
time: 09:50
text: "Aa  bagaimana rasanya?" ia sedikit terkekeh belum pernah ia tawarkan sebelumnya sesuatu yang ia buat pada seseorang.  "Aku jadi takut jika habis ini kau mengalami sakit perut" ujarnya sambil menampilkan cengiran khasnya.
tweet id: 1071235663394029569 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6909
username: @_Muzigner_
date: 2018-12-08
time: 09:50
text: Ya allah mules
tweet id: 1071235452819001344 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6910
username: @resnatary
date: 2018-12-08
time: 09:48
text: Kalo sarapan mules y? — Itu dulu. 
tweet id: 1071234980905451520 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6911
username: @jinkiofalltrade
date: 2018-12-08
time: 09:47
text: Pengen 

6948
username: @Tweet_Dokter
date: 2018-12-08
time: 09:00
text: Agar tidak sakit perut  hindari buah-buahan asam untuk jus di pagi hari... #kesehatan
tweet id: 1071223049834258432 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6949
username: @lemonenamhari
date: 2018-12-08
time: 09:00
text: Aduh tiba tiba mules :(
tweet id: 1071223022785347585 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6950
username: @aboingsyah
date: 2018-12-08
time: 09:00
text: Bisa mules?
tweet id: 1071222896867983360 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6951
username: @annisakmaulida
date: 2018-12-08
time: 08:57
text: Ya Alloh pagi pagi udah ngakak sampe sakit perut  Berkah banget dah hari sabtu ku hahah
tweet id: 1071222245232586752 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6952
username: @HidayatieAmirah
date: 2018-12-08
time: 08:57
text: Fhm dok perasaan sakit perut    tp dokle tido sbb sakit dada?!!!!!
tweet id: 1071222148906180608 
place id:  
daerah: 

6987
username: @jgwooct
date: 2018-12-08
time: 08:32
text: shempack…. mules nju gakuat…  matiin ase lontong      astatang nangis sekebon nih gua //
tweet id: 1071215920352096258 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6988
username: @latjibolala
date: 2018-12-08
time: 08:32
text: terima kasih! ta-tapi itu ga sakit perut apa pagi-pagi makan itu???
tweet id: 1071215899447676928 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6989
username: @S0NG_YUQI
date: 2018-12-08
time: 08:32
text: nanti sakit perut makan nasi padang jam segini/?  Gausah makan aja!
tweet id: 1071215831944617984 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6990
username: @fanxychillin
date: 2018-12-08
time: 08:32
text: Semoga kamu ga mules nder :(
tweet id: 1071215799283707904 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

6991
username: @kirantiiii
date: 2018-12-08
time: 08:31
text: diare kan bukan karena makan pedes doang bukan? siapa tau makanannya gak sehat
tweet id: 

7023
username: @alexsima9
date: 2018-12-08
time: 07:59
text: Kita scan saja satu-persatu mukanya... 2019GP kita garap rame2. dari pada mengharap yg sekarang dgn Moto #Suka2Kita ... Mules
tweet id: 1071207705564524544 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7024
username: @parkim_magician
date: 2018-12-08
time: 07:57
text: Sumpah ini mules karena jam segini tugas gue belum kelar atau karena bentar lqgi bakal ketemu lagi sama day6:(((
tweet id: 1071207209801895941 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7025
username: @rafikatrst
date: 2018-12-08
time: 07:55
text: Samaa sakit perut jugaa :(
tweet id: 1071206573031088128 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7026
username: @angalace
date: 2018-12-08
time: 07:55
text: Nyesel makan pedes skrg mau tes tp mules fak
tweet id: 1071206526193360896 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7027
username: @kucridkacrud
date: 2018-12-08
time: 07:53
text: Tàpi kalo mules kan bisa du

7062
username: @Jaaaaanneee
date: 2018-12-08
time: 07:12
text: Pagi pagi sakit perut ni mmg tak chill eeeeee
tweet id: 1071195794848473088 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7063
username: @pengalunmelodi
date: 2018-12-08
time: 07:12
text: Yaa Allah  bikin aku jangan sakit perut pls
tweet id: 1071195789374840832 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7064
username: @MissNyemil
date: 2018-12-08
time: 07:10
text: Yah udah rapi rapi  mau saraan tapi takut muleh nih ... Tenang ajaah MissNyemil punya cemilan anti mules
tweet id: 1071195325786808321 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7065
username: @isavailable1
date: 2018-12-08
time: 07:05
text: Dari tadi malam setiap makan  langsung diare
tweet id: 1071194000462307328 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7066
username: @kerta_jwardhana
date: 2018-12-08
time: 07:05
text: Pagi ² mules  ehbditambah liat komennya ini 
tweet id: 1071193937979686912 
place id:  
da